In [1]:
# !wget https://huggingface.co/datasets/malaysia-ai/crawl-astroawani/resolve/main/berita-bisnes-v2.json.nested
# !wget https://huggingface.co/datasets/malaysia-ai/crawl-astroawani/resolve/main/berita-politik-v2.json.nested

In [2]:
# !wget https://huggingface.co/datasets/malaysia-ai/crawl-astroawani/resolve/main/berita-dunia-v2.json.nested
# !wget https://huggingface.co/datasets/malaysia-ai/crawl-astroawani/resolve/main/berita-malaysia-v2.json.nested
# !wget https://huggingface.co/datasets/malaysia-ai/crawl-astroawani/resolve/main/berita-sukan-v2.json.nested
# !wget https://huggingface.co/datasets/malaysia-ai/crawl-astroawani/resolve/main/berita-teknologi-v2.json.nested
# !wget https://huggingface.co/datasets/malaysia-ai/crawl-astroawani/resolve/main/gaya-hidup-v2.json.nested

In [4]:
import malaya
import mp
from malaya_speech.tts import load_text_ids

malaya

/home/husein/ssd4/malaya/malaya/tokenizer.py:214: FutureWarning: Possible nested set at position 3397
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))
/home/husein/ssd4/malaya/malaya/tokenizer.py:214: FutureWarning: Possible nested set at position 3927
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))
`openai-whisper` is not available, native whisper processor is not available, will use huggingface processor instead.
/home/husein/ssd4/malaya/malaya/normalizer/rules.py:217: FutureWarning: Possible nested set at position 42
  k.lower(): re.compile(_expressions[k]) for k, v in _expressions.items()
/home/husein/ssd4/malaya/malaya/normalizer/rules.py:217: FutureWarning: Possible nested set at position 3
  k.lower(): re.compile(_expressions[k]) for k, v in _expressions.items()


<module 'malaya' from '/home/husein/ssd4/malaya/malaya/__init__.py'>

In [14]:
def loop(texts):
    texts, _ = texts
    normalizer = malaya.normalize.normalizer()
    processed_text = []
    for t in tqdm(set(texts)):
        t = t.strip()
        string = normalizer.normalize(t, normalize_hingga = False, normalize_text = True, normalize_word_rules = False, normalize_time = True, normalize_cardinal = True)
        t_ = string['normalize']
        if '--' in t_:
            continue
        if '~' in t_:
            continue
        processed_text.append({
            'original': t,
            'normalized': t_
        })
    return processed_text

In [9]:
loop((['helo'], 0))

100%|████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.34it/s]


['helo']

In [10]:
import json
import os
from tqdm import tqdm
from bs4 import BeautifulSoup
from glob import glob

files = glob('*.json.nested')[::-1]
files

['berita-dunia-v2.json.nested',
 'berita-malaysia-v2.json.nested',
 'berita-bisnes-v2.json.nested',
 'berita-hiburan-v2.json.nested',
 'gaya-hidup-v2.json.nested',
 'berita-politik-v2.json.nested',
 'berita-teknologi-v2.json.nested',
 'berita-sukan-v2.json.nested']

In [ ]:
!rm news-*-v2.json

In [21]:
for f in files:
    left = f.split('-v2')[0]
    new_f = f'news-{left}-v2.json'
    if os.path.exists(new_f):
        continue
    with open(f) as fopen:
        data = json.load(fopen)
    
    texts = []

    for d in tqdm(data):
        if 'r' not in d:
            continue
        if 'title' not in d['r']['response']:
            continue
        if len(d['r']['response']['title']):
            texts.append(d['r']['response']['title'])
        if len(d['r']['response']['description']):
            texts.append(d['r']['response']['description'])

        s = d['r']['response']['articleBody']
        soup = BeautifulSoup(s)
        s = BeautifulSoup(soup.text).text
        splitted = s.split('\n')
        for s in splitted:
            if len(s.strip()) < 10:
                continue
            if '/' in s:
                continue
            if '--' in s:
                continue
            if '~' in s:
                continue
            texts.append(s)
    
    processed_text = mp.multiprocessing(texts, loop, cores = 6)
    
    with open(new_f, 'w') as fopen:
        json.dump(processed_text, fopen)

 64%|██████████████████████████████████████████▌                        | 14792/23272 [00:40<00:40, 210.80it/s]/home/husein/ssd4/malaya/malaya/normalizer/rules.py:445: FutureWarning: Possible nested set at position 3
  if re.findall(_expressions['ic'], word_lower):
 58%|██████████████████████████████████████▉                            | 13721/23577 [00:40<00:36, 269.98it/s]/home/husein/ssd4/malaya/malaya/normalizer/rules.py:445: FutureWarning: Possible nested set at position 3
  if re.findall(_expressions['ic'], word_lower):
 69%|██████████████████████████████████████████████▏                    | 16245/23577 [00:50<00:28, 256.90it/s]/home/husein/ssd4/malaya/malaya/normalizer/rules.py:445: FutureWarning: Possible nested set at position 3
  if re.findall(_expressions['ic'], word_lower):
 75%|██████████████████████████████████████████████████▏                | 17371/23164 [00:52<00:12, 453.96it/s]/home/husein/ssd4/malaya/malaya/normalizer/rules.py:445: FutureWarning: Possible nested set

In [22]:
not_same = []
for p in processed_text:
    if p['original'] != p['normalized']:
        not_same.append(p)
        
len(not_same)

115167

In [17]:
!rm news-*-v2.json

In [24]:
glob('news-*-v2.json')

['news-berita-dunia-v2.json',
 'news-berita-politik-v2.json',
 'news-berita-bisnes-v2.json',
 'news-berita-sukan-v2.json',
 'news-berita-hiburan-v2.json',
 'news-gaya-hidup-v2.json',
 'news-berita-malaysia-v2.json',
 'news-berita-teknologi-v2.json']

In [25]:
from huggingface_hub import HfApi
api = HfApi()

for f in glob('news-*-v2.json'):
    print(f)
    api.upload_file(
        path_or_fileobj=f,
        path_in_repo=f"texts/{f}",
        repo_id="mesolitica/TTS",
        repo_type="dataset",
    )

news-berita-dunia-v2.json


news-berita-dunia-v2.json:   0%|          | 0.00/29.5M [00:00<?, ?B/s]

news-berita-politik-v2.json


news-berita-politik-v2.json:   0%|          | 0.00/48.8M [00:00<?, ?B/s]

news-berita-bisnes-v2.json


...

news-gaya-hidup-v2.json


news-gaya-hidup-v2.json:   0%|          | 0.00/30.9M [00:00<?, ?B/s]

news-berita-malaysia-v2.json


news-berita-malaysia-v2.json:   0%|          | 0.00/41.5M [00:00<?, ?B/s]

news-berita-teknologi-v2.json
